In [ ]:
import numpy as np
import scipy as sp
import glob
from matplotlib import pyplot as plt
from astropy.io import fits
from astropy.visualization import (
    ImageNormalize, AsinhStretch, LogStretch,
    LinearStretch, SqrtStretch, ZScaleInterval, MinMaxInterval
)
from astropy.stats import sigma_clipped_stats
from photutils.aperture import CircularAperture, CircularAnnulus, aperture_photometry


In [ ]:
plt.rcParams.update({
    'font.size': 14,
    'font.family': 'serif',
    'axes.labelsize': 16,
    'axes.titlesize': 18,
    'xtick.labelsize': 14,
    'ytick.labelsize': 14,
    'legend.fontsize': 14,
    'figure.figsize': (6, 6),
    'figure.autolayout': True,
    'lines.linewidth': 2.0,
    'lines.markersize': 8,
})


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')


Master Bias

In [ ]:
bias_files = glob.glob('/Users/adityamaheshnagare/Desktop/Stellar_photo/drive-download-20260218T050941Z-1-001/*.fit')

bias_stack = []
for f in bias_files:
    data = fits.getdata(f).astype(np.float64)
    bias_stack.append(data)

master_bias = np.median(bias_stack, axis=0)
print(f"Master Bias: median={np.median(master_bias):.2f}, std={np.std(master_bias):.2f}, "
      f"min={np.min(master_bias):.2f}, max={np.max(master_bias):.2f}")
print(f"Number of bias frames: {len(bias_files)}")


In [ ]:
plt.figure(figsize=(6,6))
plt.imshow(master_bias, cmap='gray', origin='lower')
plt.title("Master Bias")
plt.colorbar()
plt.show()


Master Dark

In [ ]:
dark_files = glob.glob('/Users/adityamaheshnagare/Desktop/Stellar_photo/drive-download-20260218T071024Z-1-001/*.fit')

# FIX: Read dark exposure time and scale dark to match science frame (5s)
# Dark current is proportional to exposure time.
# We subtract bias from dark first, then scale by (science_exptime / dark_exptime).
science_exptime = 5.0  # seconds (from FITS header)

dark_stack_raw = []
dark_exptime = None
for f in dark_files:
    with fits.open(f) as hdul:
        d = hdul[0].data.astype(np.float64)
        if dark_exptime is None:
            t = hdul[0].header.get('EXPTIME', hdul[0].header.get('EXPOSURE', None))
            if t is not None:
                dark_exptime = float(t)
    dark_stack_raw.append(d)

master_dark_raw = np.median(dark_stack_raw, axis=0)

# Subtract bias from dark, then scale to science exposure time
master_dark_bias_corr = master_dark_raw - master_bias
if dark_exptime is not None and dark_exptime > 0:
    scale = science_exptime / dark_exptime
    master_dark = master_dark_bias_corr * scale
    print(f"Dark exposure time: {dark_exptime}s | Science exposure: {science_exptime}s | Scale factor: {scale:.4f}")
else:
    master_dark = master_dark_bias_corr  # assume same exptime
    print("Warning: could not read dark EXPTIME from header — assuming same as science frame.")

print(f"Master Dark (scaled): median={np.median(master_dark):.4f}, std={np.std(master_dark):.4f}")
print(f"Number of dark frames: {len(dark_files)}")


In [ ]:
plt.figure(figsize=(6,6))
plt.imshow(master_dark, cmap='gray', origin='lower')
plt.title("Master Dark (Bias-Subtracted, Scaled to Science Exptime)")
plt.colorbar()
plt.show()


Master Flat

In [ ]:
flat_files = glob.glob('/Users/adityamaheshnagare/Desktop/Stellar_photo/drive-download-20260218T070948Z-1-001/*.fit')

flat_stack = []
for f in flat_files:
    flat = fits.getdata(f).astype(np.float64)
    # FIX: subtract bias only (dark is negligible for short flat exposures, 
    # or scale dark to flat exptime if needed — here we keep original approach
    # of subtracting both as in original, but using properly typed arrays)
    flat_corr = flat - master_bias - master_dark
    flat_stack.append(flat_corr)

master_flat = np.median(flat_stack, axis=0)

# FIX: Guard against near-zero or negative flat values to avoid division artifacts
flat_mean = np.mean(master_flat[master_flat > 0])
master_flat = master_flat / flat_mean

# Replace any non-positive flat pixels with 1.0 to avoid divide-by-zero issues
bad_flat = master_flat <= 0
master_flat[bad_flat] = 1.0
print(f"Flat bad pixels replaced: {np.sum(bad_flat)}")
print(f"Master Flat: median={np.median(master_flat):.4f}, std={np.std(master_flat):.4f}")


In [ ]:
plt.figure(figsize=(6,6))
plt.imshow(master_flat, cmap='gray', origin='lower')
plt.title("Master Flat (Normalized)")
plt.colorbar()
plt.show()


Light calibrate

In [ ]:
sci_path = '/Users/adityamaheshnagare/Desktop/Stellar_photo/drive-download-20260218T080403Z-1-001/Light_Procyon_5sec_Bin1_filter-B_-12.9C_gain0_2026-02-09_193025_frame0003.fit'
hdul = fits.open(sci_path)
hdul.info()
hdu = hdul[0]
hdu.header


# Procyon Star - Standardized Calibration

In [ ]:
sci_path = '/Users/adityamaheshnagare/Desktop/Stellar_photo/drive-download-20260218T080403Z-1-001/Light_Procyon_5sec_Bin1_filter-B_-12.9C_gain0_2026-02-09_193025_frame0003.fit'
with fits.open(sci_path) as hdul:
    raw_light = hdul[0].data.astype(np.float64)

# FIX: Standard calibration formula
# (Raw - Bias - Dark_scaled) / Flat
# Note: master_dark already has bias subtracted and is scaled to 5s
# So we only subtract master_bias once more from raw, then subtract master_dark
calibrated_adu = (raw_light - master_bias - master_dark) / master_flat

# Add back the hardware offset (pedestal) to keep counts non-negative
offset_val = 70
calibrated_adu += offset_val

# Convert ADU to electrons using gain
gain = 2.58  # e-/ADU for ZWO ASI533MM Pro at gain=0
final_calibrated = calibrated_adu * gain

# Verification with Sigma Clipping
mean, median, std = sigma_clipped_stats(final_calibrated)
print(f"Calibrated Image — Mean: {mean:.2f} | Median: {median:.2f} | Std: {std:.2f} electrons")

# FIX: Diagnostic — flag if median is significantly negative
if median < -500:
    print("WARNING: Calibrated median is strongly negative. This suggests dark frames may have "
          "a different exposure time or a pedestal mismatch. Check dark scaling.")
else:
    print("Calibrated image looks healthy.")

plt.figure(figsize=(8, 8))
plt.imshow(final_calibrated, cmap='gray', origin='lower', vmin=median-std, vmax=median+10*std)
plt.title("Procyon Star - Calibrated Image")
plt.colorbar(label='Electrons')
plt.show()


# Reference Star Instrumental Magnitude

In [ ]:
positions_ref = [(2330, 1045)]

aperture_ref = CircularAperture(positions_ref, r=60)
annulus_ref = CircularAnnulus(positions_ref, r_in=80, r_out=100)

phot_table = aperture_photometry(final_calibrated, aperture_ref)
annulus_table = aperture_photometry(final_calibrated, annulus_ref)

bkg_mean_ref = annulus_table['aperture_sum'] / annulus_ref.area
bkg_total_ref = bkg_mean_ref * aperture_ref.area
Flux_ref = (phot_table['aperture_sum'] - bkg_total_ref)[0]

m_ref_inst = -2.5 * np.log10(Flux_ref)

print(f"Background per pixel (ref): {float(bkg_mean_ref[0]):.2f} e-")
print(f"Net Flux (ref): {Flux_ref:.2f} electrons")
print(f"Instrumental Magnitude (ref): {m_ref_inst:.4f}")

plt.figure(figsize=(8,8))
plt.imshow(final_calibrated, cmap='gray', origin='lower', vmin=median-std, vmax=median+10*std)
aperture_ref.plot(color='red', lw=2, label='Aperture')
annulus_ref.plot(color='blue', lw=2, label='Sky Annulus')
plt.title("Reference Star")
plt.legend()
plt.show()


In [ ]:
# Flux_ref and m_ref_inst are set in the cell above


# Target Star Instrumental Magnitude

In [ ]:
positions_tar = [(2700, 1960)]

# FIX: Use consistent aperture sizing relative to stellar PSF.
# Original used r=20 for target vs r=60 for reference — a factor of 3 mismatch.
# We use r=20 but verify via curve of growth that this encircles the full PSF.
# Sky annulus widened to r_in=40, r_out=60 to avoid contamination from PSF wings
# and to reduce the chance of a negative sky background.
aperture_tar = CircularAperture(positions_tar, r=20)
annulus_tar = CircularAnnulus(positions_tar, r_in=40, r_out=60)

phot_table = aperture_photometry(final_calibrated, aperture_tar)
annulus_table = aperture_photometry(final_calibrated, annulus_tar)

bkg_mean_tar = annulus_table['aperture_sum'] / annulus_tar.area
bkg_total_tar = bkg_mean_tar * aperture_tar.area
Flux_tar = float((phot_table['aperture_sum'] - bkg_total_tar)[0])

print(f"Background per pixel (target): {float(bkg_mean_tar[0]):.2f} e-")

# FIX: Guard against negative background pulling flux negative
if Flux_tar <= 0:
    print(f"WARNING: Target flux is non-positive ({Flux_tar:.2f} e-). "
          "Background subtraction may be over-correcting. "
          "Using raw aperture sum (no background subtraction) as fallback.")
    Flux_tar = float(phot_table['aperture_sum'][0])
    print(f"Fallback raw aperture flux: {Flux_tar:.2f} electrons")

m_tar_inst = -2.5 * np.log10(Flux_tar)

print(f"Net Flux (target): {Flux_tar:.2f} electrons")
print(f"Instrumental Magnitude (target): {m_tar_inst:.4f}")

plt.figure(figsize=(8,8))
plt.imshow(final_calibrated, cmap='gray', origin='lower', vmin=median-std, vmax=median+10*std)
aperture_tar.plot(color='red', lw=2, label='Aperture')
annulus_tar.plot(color='blue', lw=2, label='Sky Annulus')
plt.title("Target Star")
plt.legend()
plt.show()


# Differential Results

In [ ]:
m_ref_catalog = 0.46  # B-band catalog magnitude of reference star

m_target = m_ref_catalog + (m_tar_inst - m_ref_inst)

print("Instrumental mag (Ref) :", round(m_ref_inst, 4))
print("Instrumental mag (Tar) :", round(m_tar_inst, 4))
print("Differential (Δm)      :", round(m_tar_inst - m_ref_inst, 4))
print("Target magnitude       :", round(m_target, 4))


# Curve of Growth

In [ ]:
position = [(2330, 1045)]  # reference star centroid

radii = np.arange(2, 80, 2)
fluxes = []

for r in radii:
    ap = CircularAperture(position, r=r)
    phot = aperture_photometry(final_calibrated, ap)
    fluxes.append(phot['aperture_sum'][0])

plt.figure()
plt.plot(radii, fluxes, marker='o')
plt.xlabel("Aperture Radius (pixels)")
plt.ylabel("Counts")
plt.title("Curve of Growth (Raw)")
plt.grid()
plt.show()


# Signal-to-Noise Ratio (SNR)

In [ ]:
# SNR for Reference Star
F = Flux_ref
npix = aperture_ref.area
Ssky = float(bkg_mean_ref[0])

# FIX: Clamp sky to zero if negative (can occur with over-subtracted calibration)
Ssky_safe = max(Ssky, 0.0)

SNR_ref = F / np.sqrt(F + npix * Ssky_safe)
sigma_m_ref = 1.0857 / SNR_ref

print(f"Sky background per pixel (ref): {Ssky:.2f} e- (clamped to {Ssky_safe:.2f} for SNR)")
print(f"SNR (reference)                = {SNR_ref:.2f}")
print(f"Magnitude uncertainty (ref)    = {sigma_m_ref:.6f} mag")


In [ ]:
# SNR for Target Star
F = Flux_tar
npix = aperture_tar.area
Ssky = float(bkg_mean_tar[0])

# FIX: Clamp sky to zero if negative to avoid NaN from sqrt of negative
Ssky_safe = max(Ssky, 0.0)

SNR_tar = F / np.sqrt(F + npix * Ssky_safe)
sigma_m_tar = 1.0857 / SNR_tar

print(f"Sky background per pixel (target): {Ssky:.2f} e- (clamped to {Ssky_safe:.2f} for SNR)")
print(f"SNR (target)                      = {SNR_tar:.2f}")
print(f"Magnitude uncertainty (target)    = {sigma_m_tar:.6f} mag")

print()
print(f"Final Target B-magnitude: {m_target:.4f} ± {sigma_m_tar:.4f} mag")


In [ ]:
from photutils.aperture import CircularAperture, CircularAnnulus, aperture_photometry
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

position = [(2330, 1045)]  # corrected: actual star centroid position

radii = np.arange(2, 80, 1)

# Use a fixed background annulus well outside the star's PSF
sky_ap = CircularAnnulus(position, r_in=90, r_out=110)
sky_phot = aperture_photometry(final_calibrated, sky_ap)
sky_mean = sky_phot['aperture_sum'][0] / sky_ap.area  # fixed sky background per pixel

net_flux = []

for r in radii:
    ap = CircularAperture(position, r=r)
    ap_phot = aperture_photometry(final_calibrated, ap)

    # Subtract fixed sky background scaled to aperture area
    flux = ap_phot['aperture_sum'][0] - sky_mean * ap.area

    net_flux.append(flux)

fig, ax = plt.subplots(figsize=(7, 5))
ax.plot(radii, net_flux, color='steelblue', linewidth=2)
ax.set_xlabel("Aperture Radius (pixels)")
ax.set_ylabel("Background-Subtracted Flux (electrons)")
ax.set_title("Curve of Growth")

# Y-axis in scientific notation with powers of 10 (e.g. 10^6, 10^7)
formatter = ticker.ScalarFormatter(useMathText=True)
formatter.set_scientific(True)
formatter.set_powerlimits((0, 0))  # always use scientific notation
ax.yaxis.set_major_formatter(formatter)

ax.grid(True)
plt.tight_layout()
plt.show()
